# Relational graphe between people

## Get data

all imports

In [195]:
import data
import pandas as pd
import csv
import networkx as nx
import plotly.express as px
import datetime
import numpy as np
import plotly.graph_objects as go


get file

In [196]:
journal = "FRANCE_SN"

d = data.open_file(data.get_data(journal))



### Create the matrice

list of people

In [197]:
#Cette liste a été réalisé après de nombreux tests fait avec tous les corpus d'articles permettant d'identifier les quelques mots clef n'étant pas des personnes s'étant glissé dans la liste de mots
DELETE_LIST = ["americain","rossignolparticipants","platiauparticipants","bresilien","president","a","republique","ag","s","fall","february","ii","jean","russie","turc","ministre","francais","malien","algerien","tchadien","prezidenta","premier"]
LIMITE = 3
dictPers = {}
for i,pers  in enumerate(d["metadata-all"]["fr"]["all"]["per"]):
    
    value = d["metadata-all"]["fr"]["all"]["per"][pers]
    normalizePers = data.normalize_name(pers)
    if normalizePers not in dictPers.keys():
        data.merge_same_name(dictPers,normalizePers,value,LIMITE)

        
#delete manually wrong info
for delete_word in DELETE_LIST:
    if delete_word in dictPers.keys():
        dictPers.pop(delete_word)
    


before = len(d["metadata-all"]["fr"]["all"]["per"])
print(f"With limite {LIMITE} before: {before}")
# print(dictPers)
print(f"With limite {LIMITE} after: {len(dictPers)}")
print(dictPers)
dfDictPers = pd.DataFrame(list(dictPers.items()), columns=['Names', 'Values'])
dfDictPers.to_csv(f"../PeopleSaves/{journal}_dictPers.csv")


With limite 3 before: 9109
With limite 3 after: 4023
{'darmanin': 559, 'paty': 982, 'estrosi': 160, 'castex': 57, 'macron': 4968, 'fouche': 50, 'sihamedi': 5, 'erdogan': 465, 'mediapart': 2, 'marianne': 255, 'boutin': 1012, 'mahomet': 198, 'darmagnac': 22, 'conte': 219, 'vardon': 248, 'clair': 131, 'sarkozy': 243, 'ghali': 135, 'derrar': 55, 'hamlat': 69, 'cherif': 46, 'ramdane': 62, 'ouane': 118, 'chaliand': 33, 'hassen': 188, 'hodcroft': 2, 'hirsch': 29, 'covid': 100, 'attal': 264, 'toussaint': 13, 'hollande': 169, 'cohen': 66, 'ikken': 7, 'gallois': 51, 'praud': 148, 'piot': 375, 'tikhanovskaia': 18, 'washington': 367, 'tagne': 299, 'jong': 208, 'balanant': 3, 'martinez': 43, 'nasrallah': 17, 'pomart': 23, 'blanquer': 163, 'villiers': 99, 'mamoun': 140, 'dupuy': 180, 'luca': 125, 'drian': 370, 'chergui': 8, 'guterrez': 25, 'ciotti': 87, 'mohamad': 41, 'noroozia': 2, 'noroozile': 2, 'aydalot': 2, 'reuters': 2, 'majid': 134, 'tolstoi': 4, 'philippe': 528, 'blanc': 70, 'sadouni': 17, '

matrice 

In [198]:
SEUIL = 4
print(f"Taille de la liste avant la suppresion des personnes à moins de {SEUIL} liens : {len(dictPers)}")
cleanList = [key for key, value in dictPers.items() if value >= SEUIL]
print(f"Taille après : {len(cleanList)}")
matrice = pd.DataFrame(0, index=cleanList, columns=cleanList)
matrice.to_csv(f"../PeopleSaves/{journal}_Matrice.csv")

Taille de la liste avant la suppresion des personnes à moins de 4 liens : 4023
Taille après : 2030


### Fill the matrice

get all the per dict from the journal

In [199]:
#delete list manually done
def dict_get_bis(journal,key):
    ret = []
    
    arts = data.open_file(data.get_data(journal))["data-all"]
    for year in arts:
        for month in arts[year]:
            for day in arts[year][month]:
                for art,i in enumerate(arts[year][month][day]):
                    dict_pers = arts[year][month][day][art][key]
                    ret.append(dict_pers)
    return ret

                    
    
allPeopleLists = dict_get_bis(journal,"per")
print(allPeopleLists)
allPeopleDict = {key:v for key,v in enumerate(allPeopleLists)}
print(f"ICI : {allPeopleDict}")
print(d["metadata-all"]["fr"]["all"]["num"])
print(len(allPeopleDict))
allPeopleDf = pd.DataFrame.from_dict(allPeopleDict,orient="index")
allPeopleDf.to_csv(f"../PeopleSaves/{journal}_allPeopleDf.csv")    

[{'Gérald Darmanin': 2}, {}, {}, {'Samuel Paty': 2}, {}, {}, {'Christian Estrosi': 3}, {}, {'Jean Castex': 2, 'M. Castex': 2}, {}, {'Emmanuel Macron': 2}, {'Emmanuel Macron': 2, 'Louis Fouché': 2, 'Président': 2}, {'Idriss Sihamedi': 5, 'Samuel Paty': 2, 'Erdogan': 3}, {'Brahim A.': 2}, {}, {}, {'Médiapart': 2}, {}, {'Marianne': 2}, {'Dominique Boutin': 28, 'Président': 2, 'Macron': 2}, {'Emmanuel Macron': 2, 'Mahomet': 2, 'Macron': 2, 'Samuel Paty': 2}, {'Marie d’Armagnac': 7, 'Giuseppe Conte': 3}, {}, {}, {'Gérald Darmanin': 3, 'Philippe Vardon': 2, 'Frédéric Saint Clair': 4, 'Nicolas Sarkozy': 3}, {'Iyad Ag Ghali': 2, 'Mustapha Derrar': 5, 'Ramdane Hamlat': 3, 'Abdelhamid Larbi Chérif': 3, 'Ramdane': 3, 'Moctar Ouane': 2}, {'Gérard Chaliand': 2}, {}, {'Béchir Ben Hassen': 2}, {'Recep Tayyip Erdogan': 2}, {}, {'Dr Emma Hodcroft': 2}, {}, {}, {}, {'Martin Hirsch': 2, 'Covid': 2}, {'Gabriel Attal': 3, 'Toussaint': 2}, {'François Hollande': 2}, {}, {'Maître Errol Cohen': 3}, {}, {}, {},

get the matrice and fill it 

In [200]:
matriceFromCsv = pd.read_csv(f"../PeopleSaves/{journal}_Matrice.csv")
clearList = matriceFromCsv.columns[1:]
indexDict = {key: value for value,key in enumerate(matriceFromCsv[cleanList])}
reverseDict = {value: key for key,value in indexDict.items()}
allPeopleDfFromCsv = pd.read_csv(f"../PeopleSaves/{journal}_AllPeopleDf.csv")
allPeopleDictFromCsv = allPeopleDfFromCsv.to_dict(orient="index")
cpt = 0


#Create a set of normalized person names for faster lookup
for article in allPeopleDictFromCsv.values():
    print(f"{cpt/len(allPeopleDictFromCsv)*100}%")
    normalized_names = {data.normalize_name(name) for name,value in article.items() if value>0 and name != "Unnamed: 0"}
    # print(clearList)
    for pers in normalized_names:
        others = [personne for personne in normalized_names if personne != pers]
        # print(f"La personne est  :  {pers}, les autres sont : {others}")
        for other in others:
            closePers = data.get_merge_name(clearList, pers)
            # print(f"La personne de la liste clearList se rapprochant le plus de {pers} est : {closePers}")
            closeOther = data.get_merge_name(clearList, other)
            # print(f"La personne de la liste clearList se rapprochant le plus de {other} est : {closeOther}")
            # print(f"La valeur de la matrice se trouvant à l'intersection {matriceFromCsv.at[indexDict[closePers],closeOther]}")
            matriceFromCsv.at[indexDict[closePers],closeOther] += 1
    cpt +=1
    
matriceFromCsv.to_csv(f"../PeopleSaves/{journal}_FinalMatrice.csv")








0.0%
0.008996041741633681%
0.017992083483267363%
0.026988125224901044%
0.035984166966534725%
0.04498020870816841%
0.05397625044980209%
0.06297229219143577%
0.07196833393306945%
0.08096437567470313%
0.08996041741633681%
0.0989564591579705%
0.10795250089960418%
0.11694854264123786%
0.12594458438287154%
0.13494062612450522%
0.1439366678661389%
0.15293270960777258%
0.16192875134940626%
0.17092479309103994%
0.17992083483267363%
0.1889168765743073%
0.197912918315941%
0.20690896005757467%
0.21590500179920835%
0.22490104354084203%
0.23389708528247571%
0.2428931270241094%
0.2518891687657431%
0.26088521050737673%
0.26988125224901044%
0.2788772939906441%
0.2878733357322778%
0.2968693774739115%
0.30586541921554516%
0.3148614609571789%
0.3238575026988125%
0.33285354444044624%
0.3418495861820799%
0.35084562792371354%
0.35984166966534725%
0.3688377114069809%
0.3778337531486146%
0.38682979489024827%
0.395825836631882%
0.4048218783735157%
0.41381792011514934%
0.42281396185678305%
0.4318100035984167%
0.

test heatmap

In [201]:
# finalOne = pd.read_csv(f"../PeopleSaves/{journal}_FinalMatrice.csv",index_col=0)
# finalOne.index = finalOne.index.map(reverseDict)
# fig = px.imshow(finalOne.values,
#                 labels=dict(x="Columns", y="Rows", color="Distance"),
#                 x=finalOne.columns,
#                 y=finalOne.index,
#                 color_continuous_scale='viridis')

# fig.update_layout(title_text='Heatmap of the Matrix', xaxis_nticks=len(finalOne.columns),yaxis_nticks=len(finalOne.columns))

# fig.show()

Transformation en graphe

In [202]:
finalOne = pd.read_csv(f"../PeopleSaves/{journal}_FinalMatrice.csv", index_col=1)
G = nx.from_pandas_adjacency(finalOne)
#Delete all nodes without link
isolated_nodes = list(nx.isolates(G))
G.remove_nodes_from(isolated_nodes)
limite = 25
nodes_to_remove = [node for node, degree in dict(G.degree()).items() if degree < limite]
G.remove_nodes_from(nodes_to_remove)
nx.write_gml(G,f"../PeopleSaves/{journal}_graphe.gml") 


# With only the ten most important person (CETTE PARTIE N'A FINALEMENT PAS ETE UTILISE CAR PAS PERTINENTE)

## get the top 10 

In [ ]:
# dfDictPers = pd.read_csv(f"../PeopleSaves/{journal}_DictPers.csv")
# dictPers = dfDictPers.set_index('Names').to_dict()["Values"]
# print(dictPers)
# top10_dict = {}
# for name,value in dictPers.items():

#     if len(top10_dict)<10:
#         top10_dict[name] = value
#     else:
#         min_key = min(top10_dict, key=top10_dict.get)
#         min_value = min(top10_dict.values())
#         if value > min_value:
#             del top10_dict[min_key]
#             top10_dict[name] = value
    
# names = list(top10_dict.keys())
# values = list(top10_dict.values())
# top10_df = pd.DataFrame({'Names':names,'Values': values})  

# print(top10_df["Names"])
# print(list(top10_df["Names"]))

## Create the empty matrice

In [ ]:
# print(names)
# print(cleanList)

# matrix_df  = pd.DataFrame(0,index = names, columns = cleanList)
# print(matrix_df)
# matrix_df.to_csv(f"../PeopleSaves/{journal}_Top10_Matrice.csv")

## fill the matrice

In [ ]:
# matriceTop10FromCsv = pd.read_csv(f"../PeopleSaves/{journal}_Top10_Matrice.csv",index_col=0)
# clearList = matriceFromCsv.columns[1:]
# indexDict = {key: value for value,key in enumerate(matriceFromCsv[cleanList])}
# # reverseDict = {value: key for key,value in indexDict.items()}
# allPeopleDfFromCsv = pd.read_csv(f"../PeopleSaves/{journal}_AllPeopleDf.csv")
# allPeopleDictFromCsv = allPeopleDfFromCsv.to_dict(orient="index")
# cpt = 0

# print(matriceTop10FromCsv)
# for article in allPeopleDictFromCsv.values():
#     print(f"{cpt/len(allPeopleDictFromCsv)*100}%")
#     normalized_names = {data.normalize_name(name) for name,value in article.items() if value>0 and name != "Unnamed: 0"}
#     # print(clearList)
#     for pers in normalized_names:
#         # print(f"La personne est  :  {pers}, les autres sont : {others}")
#         closePers = data.get_merge_name(clearList, pers)
#         print(f"Pers : {closePers}")
#         for name in names:
#             if name in normalized_names:
#                 closename = data.get_merge_name(clearList, name)
#                 print(f"top10:  {name}")
#                 # print(f"La personne de la liste clearList se rapprochant le plus de {pers} est : {closePers}")
#                 # print(f"La personne de la liste clearList se rapprochant le plus de {other} est : {closeOther}")
#                 # print(f"La valeur de la matrice se trouvant à l'intersection {matriceFromCsv.at[indexDict[closePers],closeOther]}")
#                 print(closename)
#                 matriceTop10FromCsv.loc[closename,closePers] += 1
#     cpt +=1
    
# matriceTop10FromCsv.to_csv(f"../PeopleSaves/{journal}_Top10_FinalMatrice.csv")

## Into graph

In [ ]:
# finalOneTop10 = pd.read_csv(f"../PeopleSaves/{journal}_Top10_FinalMatrice.csv", index_col=0)
# # print(finalOne)
# GTop10 = nx.DiGraph()

# # Ajouter les nœuds correspondant aux colonnes du DataFrame
# for column in matrix_df.columns:
#     GTop10.add_node(column)

# # Ajouter les arcs pondérés (edges) au graphe en utilisant les valeurs de la matrice
# for i in range(len(matrix_df)):
#     for j in range(len(matrix_df.columns) - 1):
#         if i != j:  # Éviter de créer des arcs du nœud à lui-même
#             weight = matrix_df.iloc[i, j]  # Utiliser la valeur de la matrice comme poids
#             GTop10.add_edge(
#                 matrix_df.columns[i],
#                 matrix_df.columns[j],
#                 weight=str(weight)  # Convertir le poids en chaîne de caractères
#             )

# nx.write_gml(GTop10, f"../PeopleSaves/{journal}_Top10_graphe.gml")